# Full World Model

This shit doesn't use any q-value neither the policies(gradient policy etc..).It just learns the dynamic of the environment.


*   State Transition - If i do action A, what happens to the state?
*   Rewards - If i do action A in state S , what reward will i get?

Tegeed predict hiisen ur dungiin loss-oor optimize hiigdeed yvna.Predicted value-g real outcomes-toi compare hiine.

The agent can simulate a sequence of actions in the world model and look ahead to see which sequence of actions leads to the most reward.



In [10]:
!pip install gymnasium
!pip install swig
!pip install "gymnasium[box2d]"

In [21]:
import numpy as np
import torch
import gymnasium as gym
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

vision_model_hidden_dim=255
memory_model_hidden_dim = 255
controller_hidden_dim = 255
action_dim=3
learning_rate = 1e-4
num_episodes=500

## Building the Vision Model(Convo Neural Network)

In [16]:
class VisionModel(nn.Module):
    def __init__(self):
        super(VisionModel,self).__init__()
        self.conv1 = nn.Conv2d(3,32,kernel_size=8,stride=4)
        self.conv2 = nn.Conv2d(32,64,kernel_size=4,stride=2)
        self.conv3 = nn.Conv2d(64,64,kernel_size=3,stride=1)
        self.fc = nn.Linear(64*7*7,vision_model_hidden_dim)
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0),-1)  # fully connected layer-too zoriulaad flatten shaaj baina.
        x = F.relu(self.fc(x))
        return x


## Building the Memory Model(Recurrent Neural Network)

In [17]:
class MemoryModel(nn.Module):
    def __init__(self):
        super(MemoryModel,self).__init__()
        self.lstm = nn.LSTM(input_size = vision_model_hidden_dim,hidden_size=memory_model_hidden_dim) # long short term memory

    def forward(self,x,hidden_state):
        x,hidden_state = self.lstm(x.unsqueeze(0),hidden_state)
        return x.squeeze(0),hidden_state


## Building the Controller Model(Fully Connected Neural Network)

In [18]:
class Controller(nn.Module):
    def __init__(self):
        super(Controller,self).__init__()
        self.fc = nn.Linear(memory_model_hidden_dim,action_dim)
    def forward(self,x):
        x = self.fc(x)
        return x

## PreProcessing the states

In [19]:
def preprocess_state(state):
    state = torch.from_numpy(state).float() / 255.0   # pixel value-g between 0-1 bolgoj normalize hiij baina.
    state = state.permute(2, 0, 1)  # Convert (height, width, channels) to (channels, height, width)  bairiig ni huurhun soliod ugchij baigaa.
    return state.unsqueeze(0)  # Add batch dimension


In [20]:
 # Setting up the environment
env = gym.make("CarRacing-v3")
 # Creating the models
vision_model = VisionModel()
memory_model = MemoryModel()
controller = Controller()


# Creating the optimizers
vision_optimizer = optim.Adam(vision_model.parameters(), lr=learning_rate)
memory_optimizer = optim.Adam(memory_model.parameters(), lr=learning_rate)
controller_optimizer = optim.Adam(controller.parameters(), lr=learning_rate)


# Implementing the Training Loop

for episode in range(num_episodes):
    state = env.reset()
    hidden_state = None
    done = False
    episode_reward = 0
    while not done:
        state = preprocess_state(state)
        with torch.no_grad():
            vision_output = vision_model(state)
            memory_output, hidden_state = memory_model(vision_output, hidden_state or (torch.zeros(1, 1, memory_model_hidden_dim), torch.zeros(1, 1, memory_model_hidden_dim)))
            action = controller(memory_output).squeeze(0)
        next_state, reward, done, _ = env.step(action.item())  # Convert action to scalar
        state = next_state
        episode_reward += reward
    print(f"Episode {episode}: Total Reward: {episode_reward}")

TypeError: expected np.ndarray (got tuple)